In [ ]:
# !wget https://www.cs.ccu.edu.tw/~wtchu/projects/Weather/Image.zip
# !unzip Image.zip
# !mv Image weather_dataset

# Uncomment to run on Colab 

In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [ ]:
# !pip install pyunpack
# !pip install patool
# from pyunpack import Archive

# Archive('/content/gdrive/My Drive/Colab Notebooks/Project/Image.rar').extractall('./')

# Split Dataset to Train, Val, Test

In [ ]:
!pip install split-folders

In [ ]:
import split_folders

In [ ]:
split_folders.ratio("Image", output="weather_dataset", seed=1337, ratio=(0.7,0.15,0.15))

In [ ]:
# Uncomment to run on colab, Change paths.

# !cp gdrive/My\ Drive/Colab\ Notebooks/Project/*.sh  .
# !cp gdrive/My\ Drive/Colab\ Notebooks/Project/*.py  .
# !chmod +x augment_images.sh
# !./augment_images.sh "./weather_dataset/train/"

In [ ]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import random_split as rsplit
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import os
import torchvision.models

torch.manual_seed(1)

In [ ]:
def get_data_loader(batch_size=8, sets=["train", "val", "test"], tlearning=""):
    data_loader = []
    
    data_transform = transforms.Compose( [transforms.CenterCrop(224), transforms.Resize((224,224)) , transforms.ToTensor()])
    
    # Load dataset based on the passed in folder name & whether it is transfer learning or not.
    for s in sets:
        if tlearning == "anet":
            print("\n #### Loading AlexNet Features ####\n")
            data_set  = torchvision.datasets.DatasetFolder("weather_dataset/{}".format("anet_{}".format(s)),  
                                                           loader=torch.load, extensions=('.tensor'))
        elif tlearning =="vgg16":
            print("\n #### Loading VGG16 Features ####\n")
            data_set  = torchvision.datasets.DatasetFolder("weather_dataset/{}".format("vgg16_{}".format(s)), 
                                                          loader=torch.load, extensions=('.tensor'))
        else:
            data_set = ImageFolder("weather_dataset/{}".format(s),
                               transform=data_transform)
        
        print("Complete Dataset:\n ", data_set)
        data_loader.append(torch.utils.data.DataLoader(data_set, batch_size=batch_size, shuffle=True))
    
    return data_loader

In [ ]:
def get_model_path(name, bs, lr, data_set, total_epoch, epoch, csv=False, tlearning=""):
    if csv:
        model_path = "{}_BS_{}_LR_{}_dataset_{}_numepochs_{}/".format(name, bs, lr, data_set, total_epoch)
        if tlearning == "anet":
            model_path = "{}_BS_{}_LR_{}_dataset_{}_numepochs_{}/".format(name,bs, lr, "anet_{}".format(data_set),total_epoch)
        elif tlearning == "vgg16":
            model_path = "{}_BS_{}_LR_{}_dataset_{}_numepochs_{}/".format(name,bs, lr, "vgg16_{}".format(data_set),total_epoch)
    else:
        model_path = "{}_BS_{}_LR_{}_dataset_{}_numepochs_{}/epoch_{}".format(name, bs, lr, data_set, total_epoch,epoch)
        if tlearning == "anet":
            model_path = "{}_BS_{}_LR_{}_dataset_{}_numepochs_{}/epoch_{}".format(name,bs, lr, "anet_{}".format(data_set),epoch)
        elif tlearning == "vgg16":
            model_path = "{}_BS_{}_LR_{}_dataset_{}_numepochs_{}/epoch_{}".format(name,bs, lr, "vgg16_{}".format(data_set),total_epoch)    
    return model_path

In [ ]:
def get_accuracy(model, data_loader, conf_matrix=False):
    # calculate total correct predictions within the specified data_set

#     classes = ['cloudy', 'foggy', 'rain', 'snow', 'sunny', 'z-other']
    classes = ['cloudy', 'foggy', 'rain', 'snow', 'sunny']
    
    x = np.zeros(shape=(len(classes), len(classes)))
#     print(x.shape)

    l = 0
    correct, total = 0 , 0
    for images, labels in data_loader:
        images = images.cuda()
        labels = labels.cuda()
        
        outputs = model(images)
        
        pred = outputs.max(1, keepdim=True)[1]
        
        if conf_matrix:
            i = labels.view_as(pred).item()
            j = pred.item()
            x[i][j] += 1
            
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += images.shape[0]
        
        l+=1
    if conf_matrix:
        return (correct / total), x
    else:
        return (correct / total)

In [ ]:
def train_net(net, batch_size=8, lr=0.01, num_epochs=20, data_set=["train", "val"], weight_decay=0, tlearning=""):
    
    model_path=""
    
    if tlearning == "anet":
        model_path = "{}_BS_{}_LR_{}_dataset_{}_numepochs_{}/".format(net.name,batch_size, lr, "anet_{}".format(data_set[0]),num_epochs)
    elif tlearning == "vgg16":
        model_path = "{}_BS_{}_LR_{}_dataset_{}_numepochs_{}/".format(net.name,batch_size, lr, "vgg16_{}".format(data_set[0]),num_epochs)
    else:
        model_path = "{}_BS_{}_LR_{}_dataset_{}_numepochs_{}/".format(net.name,batch_size, lr, data_set[0],num_epochs)

    
    os.mkdir(model_path)
    
    
    with open('./{}network_params.txt'.format(model_path),'w') as file:    
        print(net.parameters, file=file)
    
    
    data_loader = get_data_loader(batch_size=batch_size, sets=data_set, tlearning=tlearning)
    train_loader, val_loader = data_loader
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
    
    train_acc, train_loss = [], []
    val_acc, val_loss = [], []
    iters = []
    
    for epoch in range(num_epochs):
        print("Epoch: ", epoch)
        
        batch_total =0.0
        
        # go through data loaded in batches of batch_size
        # i.e. # of data loaded / batch_size should be number of iterations
        
        j = 0
        total_loss = 0
        for i, data in enumerate(train_loader, 0):              
            images, labels = data

#             print(labels)
    
            images = images.cuda()
            labels = labels.cuda()

            outputs = net(images)
#             print(outputs.shape)
            loss = criterion(outputs, labels)
            loss.backward()
            total_loss+=float(loss)
            j+=images.shape[0]
            optimizer.step()
            optimizer.zero_grad()

            pred = outputs.max(1, keepdim=True)[1]
            
#             print(pred)
            mini_batch_correct = pred.eq(labels.view_as(pred)).sum().item()
            mini_batch_total = images.shape[0]
#             train_acc.append(mini_batch_correct / mini_batch_total)

            print("Epoch {}, Mini Batch Accuracy {:.2f}%, Loss {:.2f}%".format(epoch ,mini_batch_correct/mini_batch_total * 100 , total_loss/j * 100))
            
#             n+=1
        
        iters.append(epoch)
        train_acc.append(get_accuracy(net, train_loader))
        val_acc.append(get_accuracy(net, val_loader))
        train_loss.append(total_loss/j)
        
        print("\nEpoch {} Summary: Train Accuracy {:.2f}%, Validation Accuracy {:.2f}%, Loss {:.2f}%\n".format(epoch,
                                                                                                          train_acc[-1] * 100,
                                                                                                          val_acc[-1] * 100,
                                                                                                          total_loss/j * 100))
        
        torch.save(net.state_dict(), model_path + "epoch_{}".format(str(epoch)))           



    
    np.savetxt("{}train_acc.csv".format(model_path), train_acc)
    np.savetxt("{}train_loss.csv".format(model_path), train_loss)
    np.savetxt("{}val_acc.csv".format(model_path), val_acc)
    np.savetxt("{}iters.csv".format(model_path), iters)
    
    return iters, train_loss, val_acc, train_acc


# Network Definitions

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.name="CNN"
        self.conv1 = nn.Conv2d(3, 5, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(5, 10, 5)
        self.fc1 = nn.Linear(10 * 53 * 53, 128)
        self.fc2 = nn.Linear(128, 5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 10 * 53 * 53)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
class anet_CNN(nn.Module):
    def __init__(self):
        super(anet_CNN, self).__init__()
        self.name="anetCNN"
        self.conv1 = nn.Conv2d(256, 5, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(5*2*2, 64)
        self.fc2 = nn.Linear(64, 5)

    def forward(self, img):
        x = self.pool(F.relu(self.conv1(img)))
        x = x.view(-1, 5*2*2)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
class vgg16_CNN(nn.Module):
    def __init__(self):
        self.name="vgg16CNN"
        super(vgg16_CNN, self).__init__()
        self.conv1 = nn.Conv2d(512, 400, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(400*2*2, 64)
        self.fc2 = nn.Linear(64, 5)
        
    def forward(self, img):
        x = self.pool(F.relu(self.conv1(img)))
        x = x.view(-1, 400*2*2)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
def plot_graphs(iters, train_loss, train_acc, val_acc):
    plt.title("Training Curve")
    plt.plot(iters, train_loss, label="Train")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.show()

    plt.title("Training Curve")
    plt.plot(iters, train_acc, label="Training")
    plt.plot(iters, val_acc, label="Validation")    
    plt.xlabel("Epochs")
    plt.ylabel("Validation Accuracy")
    plt.legend(loc='best')
    plt.show()

    print("Final Training Accuracy: {}".format(train_acc[-1]))
    print("Final Validation Accuracy: {}".format(val_acc[-1]))

In [ ]:
def print_conf_matrix(conf_matrix):
    print("\n\n")

    classes = ['cloudy', 'foggy', 'rain', 'snow', 'sunny']

    for i in range(len(classes)):
        if i == 0:
            print('', end='\t')

        print(classes[i], end = '\t')

    print('\n')

    for i in range(len(classes)):

        print(classes[i], end ='\t')
        for j in range(len(classes)):
            print(conf_matrix[i][j], end='\t')
        print('\n')

In [ ]:
def save_anet_feats(sets = ["train", "val", "test"], tlearning="anet"):
    root_path = "weather_dataset/"
    classes = ['cloudy', 'foggy', 'rain', 'snow', 'sunny']
    
    pretrained_net = ""
    subpath = ""
    if tlearning == "anet":
        print("Saving as anet features")
        pretrained_net = torchvision.models.alexnet(pretrained=True)
        subpath = "anet"
    elif tlearning =="vgg16":
        print("Saving as vgg16 features")
        pretrained_net = torchvision.models.vgg16(pretrained=True)
        subpath = "vgg16"
    else:
        assert 0
    
    data_loader = get_data_loader(batch_size=1, sets=sets)
    
    for i, data in enumerate(data_loader):
        path = "{}/{}_{}".format(root_path, subpath,sets[i])
        
        if not os.path.isdir(path):
            os.mkdir(path)
            
        n = 0
        
        for images, labels in data:
            features = pretrained_net.features(images)
            features_tensor = torch.from_numpy(features.detach().numpy())
            folder = "{}/{}/".format(path, classes[labels])
            
            if not os.path.isdir(folder):
                os.mkdir(folder)
                
            torch.save(features_tensor.squeeze(0), '{}/{}.tensor'.format(folder, str(n)))
            n+=1
    
    

# Running Regular CNN

In [ ]:
net = CNN().cuda()
iters, train_loss, val_acc, train_acc = train_net(net, num_epochs=30, lr=0.01, batch_size=512, data_set=["train","val"])


In [ ]:
plot_graphs(iters, train_loss, train_acc, val_acc)
print(train_acc[-1])

In [ ]:
data_loader = get_data_loader(batch_size=1, sets=["val"])
x, y = get_accuracy(net, data_loader[0], conf_matrix=True)

In [ ]:
print(x)
print_conf_matrix(y)

# Running AlexNet 

In [ ]:
# Can save the alexnet features into your drive to avoid running this again
save_anet_feats(sets = ["train", "val", "test"],tlearning="anet") #if using alexnet transfer learning

In [ ]:
net = anet_CNN().cuda()
iters, train_loss, val_acc, train_acc = train_net(net, num_epochs=100, lr=0.001, batch_size=512, data_set=["train","val"], tlearning="anet")

In [ ]:
plot_graphs(iters, train_loss, train_acc, val_acc)
print(train_acc[-1])

In [ ]:
val_max_idx = np.argmax(val_acc)
print(val_max_idx, train_acc[val_max_idx], val_acc[val_max_idx])


In [ ]:
data_loader = get_data_loader(batch_size=1, sets=["val"], tlearning="anet")
x, y = get_accuracy(net, data_loader[0], conf_matrix=True)

In [ ]:
print(x)
print_conf_matrix(y)

# Running VGG16-Net

In [ ]:
# Can save the vgg16 features into your drive to avoid running this again
save_anet_feats(sets = ["train", "val", "test"], tlearning="vgg16")

In [ ]:
net = vgg16_CNN().cuda()
iters, train_loss, val_acc, train_acc = train_net(net, num_epochs=20, lr=0.015, batch_size=1024, data_set=["train","val"], tlearning="vgg16")

In [ ]:
plot_graphs(iters, train_loss, train_acc, val_acc)
print(train_acc[-1])

In [ ]:
val_max_idx = np.argmax(val_acc)
print(val_max_idx, train_acc[val_max_idx], val_acc[val_max_idx])

In [ ]:
data_loader = get_data_loader(batch_size=1, sets=["val"], tlearning="vgg16")
x, y = get_accuracy(net, data_loader[0], conf_matrix=True)

In [ ]:
print(x)
print_conf_matrix(y)

In [ ]:
# model_path = get_model_path(model.name, bs=256, lr=0.001, data_set="train", total_epoch=100, epoch=100,csv=True)
# state = torch.load(model_path + "epoch_100")
# model.load_state_dict(state)

# iters = np.loadtxt(open(model_path +"iters.csv", "rb"), delimiter=",")
# train_acc = np.loadtxt(open(model_path + "train_acc.csv", "rb"), delimiter=",")
# train_loss = np.loadtxt(open(model_path+ "train_loss.csv", "rb"), delimiter=",")
# val_acc = np.loadtxt(open(model_path + "val_acc.csv", "rb"), delimiter=",")

# plot_graphs(iters, train_loss, train_acc, val_acc)